# NeonPandas Example

In [1]:
import pandas as pd 
import neonpandas as npd

## Create NodeFrame

In [14]:
pets = pd.read_csv('datasets/pets.csv')

In [15]:
## can convert existing dataframe or any input data that Pandas DataFrame can accept
nodes = npd.NodeFrame(pets, id_col='name', labels={'Pet'}, lbl_col='species')
nodes

,labels,name,color,age,behavior
0,"{Dog, Pet}",Ralph,black,10.0,NaN
1,"{Cat, Pet}",Pip,yellow,6.0,good
2,"{Pig, Pet}",Babe,NaN,3.0,NaN
3,"{Fish, Pet}",Bubbles,red,NaN,acceptable
4,"{Horse, Pet}",Freckles,brown,NaN,NaN


## Create EdgeFrame

In [4]:
data = pd.read_csv('datasets/edges.csv')

In [5]:
edges = npd.EdgeFrame(data, rel_col='rel_type', 
                      src_col='src', dest_col='dest', 
                      src_id='name', dest_id='name',
                      labels={'Pet'})
edges

,rel_type,src,dest,reporter,length
0,FRIENDLY_WITH,"(s:Pet {name: ""Ralph""})","(d:Pet {name: ""Bubbles""})",Jenny,2 years
1,MEAN_TO,"(s:Pet {name: ""Pip""})","(d:Pet {name: ""Babe""})",Frank,1 month
2,MEAN_TO,"(s:Pet {name: ""Ralph""})","(d:Pet {name: ""Freckles""})",Frank,1 day
3,FRIENDLY_WITH,"(s:Pet {name: ""Freckles""})","(d:Pet {name: ""Babe""})",Tommy,10 months
4,JUST_MET,"(s:Pet {name: ""Pip""})","(d:Pet {name: ""Barney""})",Jenny,NaN


## Add Nodes & Edges to Graph

In [6]:
graph = npd.Graph(uri='bolt://localhost:7687', auth=('neo4j', 'neonpandas'))

In [ ]:
## add nodes to graph
graph.create_nodes(nodes)

In [ ]:
## add edges to graph
graph.create_edges(edges)

## Neo4j Bolt Output
![](src/bolt_vis.png)

## Matching

In [9]:
graph.match_nodes(labels={'Pet'}, id_col='name')

,labels,color,name,behavior,age
0,"{Fish, Pet}",red,Bubbles,acceptable,NaN
1,{Pet},NaN,Barney,NaN,NaN
2,"{Dog, Pet}",black,Ralph,NaN,10.0
3,"{Cat, Pet}",yellow,Pip,good,6.0
4,"{Pig, Pet}",NaN,Babe,NaN,3.0
5,"{Horse, Pet}",brown,Freckles,NaN,NaN


In [8]:
## add properties to search
graph.match_nodes(labels={'Pet'}, properties={'name': 'Ralph'})

,labels,color,name,age
0,"{Dog, Pet}",black,Ralph,10.0


## Joining
Comparing data inside and outside of Neo4j.

In [16]:
new_pets = pd.DataFrame([
    {'name': 'Betsy', 'age': 2, 'species': 'Cow'},
    {'name': 'Carrie', 'species': 'Rabbit'}
])
all_pets = pd.concat([pets, new_pets], sort=False).reset_index(drop=True)

In [20]:
# convert to nodeframe
all_pets = npd.NodeFrame(all_pets, id_col='name', lbl_col='species', labels={'Pet'})
all_pets

,labels,name,color,age,behavior
0,"{Dog, Pet}",Ralph,black,10.0,NaN
1,"{Cat, Pet}",Pip,yellow,6.0,good
2,"{Pig, Pet}",Babe,NaN,3.0,NaN
3,"{Fish, Pet}",Bubbles,red,NaN,acceptable
4,"{Horse, Pet}",Freckles,brown,NaN,NaN
5,"{Cow, Pet}",Betsy,NaN,2.0,NaN
6,"{Rabbit, Pet}",Carrie,NaN,NaN,NaN


### Semi-Join

In [22]:
graph.semi_join(all_pets, on='name', labels={'Pet'})

,labels,color,name,age,behavior
0,"{Dog, Pet}",black,Ralph,10.0,NaN
1,"{Cat, Pet}",yellow,Pip,6.0,good
2,"{Pig, Pet}",NaN,Babe,3.0,NaN
3,"{Fish, Pet}",red,Bubbles,NaN,acceptable
4,"{Horse, Pet}",brown,Freckles,NaN,NaN


### Anti-Join

In [26]:
new_pets = graph.anti_join(all_pets, on='name', labels={'Pet'})
new_pets

,labels,name,color,age,behavior
0,"{Cow, Pet}",Betsy,NaN,2.0,NaN
1,"{Rabbit, Pet}",Carrie,NaN,NaN,NaN
